In [1]:
from dataset import ConllDataset

original_file = '../data/wsj_dev.conll06.pred'
conll_dataset = ConllDataset(original_file)

In [2]:
from features import FeatureMapping

feature = FeatureMapping()
for i in range(100):
    sentence = conll_dataset[i]
    feature.get(sentence)

feature.frozen = True

In [32]:
def get_training_data(dataset, size=100):
    feature = FeatureMapping()
    y = []

    for i in range(size):
        sentence = conll_dataset[i]
        # Train extractor
        # TODO: create function that trains on training data instead of this
        true_feature = feature.get(sentence)
        arcs = conll_dataset[i].get_arcs()

        y.append([true_feature, arcs])

    feature.frozen = True
    for i, (f, arcs) in enumerate(y):
        sentence = conll_dataset[i]
        x = feature.get_permutations(sentence)

        yield x, f, arcs

In [3]:
from model import Perceptron

model = Perceptron(dim=feature.feature_count())

In [27]:
import decoder as decoder
import numpy as np
import copy

cle = decoder.CLE()

instances = list(get_training_data(conll_dataset, 100))

epochs = 1
for i in range(epochs):
    for (x, true_features, y) in instances:
        scores = model.forward(x)

        scores[:, 0] = -np.inf
        scores[np.diag_indices_from(scores)] = -np.inf

        predicted_y = cle.decode(scores)

        if y != predicted_y:
            predicted_arcs = [arc[0] for arc in predicted_y]

            sentence_copy = copy.deepcopy(sentence)
            sentence_copy.sentence['head'] = predicted_arcs
            predicted_feature = np.sum(feature.get(sentence_copy), axis=0)
            true_feature = np.sum(true_features, axis=0)

            model.weight = np.add(model.weight, true_feature - predicted_feature)

NameError: name 'get_training_data' is not defined

In [26]:
conll_dataset[0]

In [31]:
instances = list(get_training_data(conll_dataset, 100))

AttributeError: 'numpy.ndarray' object has no attribute 'get_permutations'